### DL Competition Winning Model : American_sign_language_image_classifier 

In [1]:
!pip install kaggle
!pip install keras
!pip install requests

    100% |################################| 51kB 2.3MB/s ta 0:00:011
  Running setup.py bdist_wheel for kaggle ... done
  Stored in directory: /root/.cache/pip/wheels/3b/8e/14/adcbb71d126f5fb939fec8aa81394450922c20eee876bf8882
Successfully built kaggle
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |################################| 307kB 1.7MB/s ta 0:00:01
    100% |################################| 51kB 4.9MB/s eta 0:00:01
    100% |################################| 2.8MB 238kB/s eta 0:00:01
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
print(os.getcwd())

/notebooks


In [5]:
'''
Function to get the data from the kaggle link
'''
import requests
import os
import time
def getdata(file_url, target_path_final):
    #file_url="https://www.kaggle.com/datamunge/sign-language-mnist/downloads/sign-language-mnist.zip/1"
    r=requests.get(file_url, stream=True)
    print(r)
    #target_path=os.getcwd()
    start_time=time.time()
    #target_path_final=target_path+"\\datasets\\"+"set00.tar"
    with open(target_path_final, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1048576):
            if chunk:f.write(chunk)
    end_time=time.time()
    print("time taken: {}".format(end_time-start_time))
    print("%s file is downloaded !\n"%target_path_final)
target_path=os.getcwd()
getdata("https://www.kaggle.com/datamunge/sign-language-mnist/downloads/sign-language-mnist.zip/1",target_path)

<Response [200]>
time taken: 0.0028104782104492188
/notebookssign-language-mnist.zip file is downloaded !



In [7]:
'''
Data is preesent through Local kaggle Api and accessed through the terminal iteselt [easiest way]
'''
import csv
path=os.getcwd()
#path="C:\\cygwin64\\home\\sananand\\.kaggle\\datasets\\datamunge\\sign-language-mnist\\"
file_train=path+"/sign_mnist_train.csv"
file_test=path+"/sign_mnist_test.csv"
train_data, test_data=[],[]
y_train,y_test=[],[]
with open(file_train, 'rt', encoding="utf8") as file_train:
    reader_train=csv.reader(file_train, delimiter=' ', quotechar='|')
    for idx, row in enumerate(reader_train):
        if idx!=0:
            temp=row[0].split(',')
            tempnew=[int(i) for idx,i in enumerate(temp) if idx!=0]
            y_train.append(int(temp[0]))
            train_data.append(tempnew)
with open(file_test, 'rt', encoding="utf8") as file_test:
    reader_test=csv.reader(file_test, delimiter=' ', quotechar='|')
    for idx, row in enumerate(reader_test):
        if idx!=0:
            temp=row[0].split(',')
            tempnew=[int(i) for idx,i in enumerate(temp) if idx!=0]
            y_test.append(int(temp[0]))
            test_data.append(tempnew)

In [8]:
import numpy as np
print("Number of Traning Examples", len(train_data))
print("Number of Test Examples", len(test_data))

Number of Traning Examples 27455
Number of Test Examples 7172


In [9]:
train_array=np.asarray(train_data)
test_array=np.asarray(test_data)

In [10]:
print(train_array.shape)
number_of_samples = train_array.shape[0]
height=28
width=28

(27455, 784)


In [11]:
train_array_transformed=np.reshape(train_array, (number_of_samples, height, width, -1))

In [12]:
train_array_transformed.shape

(27455, 28, 28, 1)

In [13]:
print(test_array.shape)
test_array_transformed=np.reshape(test_array, (test_array.shape[0], height, width, -1))

(7172, 784)


In [14]:
test_array_transformed.shape

(7172, 28, 28, 1)

In [16]:
!pip install tensorflow

    100% |################################| 51.1MB 13kB/s  eta 0:00:01
    100% |################################| 9.1MB 76kB/s  eta 0:00:01
    100% |################################| 3.3MB 218kB/s eta 0:00:01
    100% |################################| 92kB 6.8MB/s eta 0:00:01
    100% |################################| 573kB 1.2MB/s eta 0:00:01
    100% |################################| 7.1MB 97kB/s  eta 0:00:01
    100% |################################| 327kB 2.2MB/s eta 0:00:01
    100% |################################| 81kB 6.4MB/s eta 0:00:01
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /root/.cache/pip/wheels/4c/16/ef/e36a23f2432e9220f8845f94e2c3abd39e7d9d1cd458d3159d
  Running setup.py bdist_wheel for gast ... done
  Stored in directory: /root/.cache/pip/wheels/9a/1f/0e/3cde98113222b853e98fc0a8e9924480a3e25f1b4008cedb4f
Successfully built absl-py gast
  Found existing installation: setuptools 39.2.0
    Uninstalling setuptools-39.2.0:
      Suc

In [17]:
# define image data generator
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)

Using TensorFlow backend.


In [18]:
# normalization
X_train = train_array_transformed / 255.0
X_test = test_array_transformed / 255.0

In [19]:
from keras import models
from keras import layers
from keras.utils import np_utils
# one hot encode
y_train=np_utils.to_categorical(y_train)
y_test=np_utils.to_categorical(y_test)

In [20]:
print(y_train.shape)
number_of_classes=y_train.shape[1]

(27455, 25)


In [21]:
#model
network=models.Sequential()
network.add(layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28,28,1,)))
network.add(layers.Conv2D(64, (3,3), padding='same', activation='relu'))
network.add(layers.MaxPooling2D(pool_size=(2,2)))
network.add(layers.Dropout(0.3))
network.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
network.add(layers.MaxPooling2D(pool_size=(2,2)))
network.add(layers.Conv2D(128, (3,3), padding='same', activation='relu'))
network.add(layers.MaxPooling2D(pool_size=(2,2)))
network.add(layers.Dropout(0.3))
network.add(layers.Flatten())
network.add(layers.Dense(1500, activation='relu'))
network.add(layers.Dropout(0.5))
network.add(layers.Dense(number_of_classes, activation='softmax'))

In [22]:
network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 128)         147584    
__________

In [23]:
# model compilation
import keras.optimizers
network.compile(optimizer='adam',
               loss = 'categorical_crossentropy',
               metrics=['accuracy'])

# keras.optimizers.RMSprop(lr=5e-4)
# Fit the model
network.fit_generator(datagen.flow(X_train, y_train, batch_size=32),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=10, validation_data=(X_test, y_test),
                        callbacks=[],verbose=2)

/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if sys.path[0] == '':
/opt/conda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., callbacks=[], verbose=2, steps_per_epoch=857, epochs=10)`
  if sys.path[0] == '':


Epoch 1/10
 - 92s - loss: 1.8232 - acc: 0.4233 - val_loss: 0.3093 - val_acc: 0.8926
Epoch 2/10
 - 89s - loss: 0.3935 - acc: 0.8665 - val_loss: 0.0941 - val_acc: 0.9713
Epoch 3/10
 - 90s - loss: 0.2083 - acc: 0.9310 - val_loss: 0.0292 - val_acc: 0.9926
Epoch 4/10
 - 90s - loss: 0.1473 - acc: 0.9508 - val_loss: 0.0126 - val_acc: 0.9971
Epoch 5/10
 - 91s - loss: 0.1153 - acc: 0.9620 - val_loss: 0.0269 - val_acc: 0.9943
Epoch 6/10
 - 90s - loss: 0.0931 - acc: 0.9691 - val_loss: 0.0040 - val_acc: 0.9994
Epoch 7/10
 - 91s - loss: 0.0923 - acc: 0.9709 - val_loss: 0.0107 - val_acc: 0.9944
Epoch 8/10
 - 93s - loss: 0.0808 - acc: 0.9730 - val_loss: 0.0113 - val_acc: 0.9941
Epoch 9/10
 - 91s - loss: 0.0727 - acc: 0.9767 - val_loss: 0.0027 - val_acc: 1.0000
Epoch 10/10
 - 92s - loss: 0.0678 - acc: 0.9797 - val_loss: 0.0135 - val_acc: 0.9944
